In [17]:
# import necessary libraries
import requests as req
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# CityBikes

In [18]:

# define the base URL for the API and the path for data assets
base_url = "http://api.citybik.es"

# define the folder path
assets_path = '../data/'

# retrieve all network information from the API and extract only the location and href fields
res = req.get(f"{base_url}/v2/networks?fields=location,href").json()['networks']


Send a request to CityBikes for the city of your choice. 

In [19]:

# set the city for which we want to retrieve bike data
city = 'Vancouver'

# extract the href fields for all networks in the specified city
companies_href = [i['href'] for i in res if city.lower() in i['location']['city'].lower()]

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [20]:

# initialize an empty dictionary to store the bike data
data = {'Free bikes': [], 'Latitude': [], 'Longitude': [], "Empty slots": [], "Id": []}

# loop through each network in the city and retrieve the bike station data
for company_endpoint in companies_href:
    # make an API call to retrieve the station data for the current network
    stations_list = req.get(f"{base_url}{company_endpoint}?fields=stations").json()['network']["stations"]
    
    # loop through each station in the current network and extract the desired data
    for index in range(len(stations_list)):
        data["Free bikes"].append(stations_list[index]['free_bikes'])
        data["Latitude"].append(stations_list[index]['latitude'])
        data["Longitude"].append(stations_list[index]['longitude'])
        data["Empty slots"].append(stations_list[index]['empty_slots'])
        data["Id"].append(stations_list[index]['id'])

Put your parsed results into a DataFrame.

In [21]:
# create a pandas DataFrame from the extracted data
df = pd.DataFrame(data).sample(n=200)

# save the DataFrame to a CSV file
df.to_csv(assets_path + 'citybikes.csv', index=False)